# batch gd
每次需要所有数据:
![bgd](img/bgd.png)

# 每次只用一个数据来计算搜索方向
## 相比batchgd 每次都是沿着梯度方向, 随机GD可能错过最佳值 所以它的学习率应该越来越小 而不是一直不变
## 这其实就是模拟退火的思想


$$
\eta = \frac{1}{i_iters}
$$
--->
$$
\eta = \frac{a}{i_iters + b}
$$


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
m = 100000
np.random.seed = 666
x = np.random.normal(size=m)
X = x.reshape(-1, 1)
y = 4. * x + 3. + np.random.normal(0, 3, size = m)

In [3]:
def J(theta, X_b, y):
    try:
        return np.sum((y-X_b.dot(theta)) ** 2) / len(y)
    except:
        return float("inf")
    
def dJ(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2./ len(y)


def gradient_descent(X_b, y, initial_theta, eta, n_iters=1e4, epsilon=1e-8):
    theta = initial_theta
    cur_iter = 0
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if (abs(J(theta, X_b,y) - J(last_theta, X_b, y)) < epsilon):
            break
        cur_iter +=1
    
    return theta
    

In [4]:
%%time
X_b = np.hstack((np.ones((len(X), 1)), X))
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
theta = gradient_descent(X_b, y, initial_theta, eta)

Wall time: 729 ms


In [5]:
theta

array([2.98805989, 3.99007309])

## 随机梯度下降

In [6]:
def dJ_sgd(theta, X_b_i, y_i):
    # X_b_i 代表第i个样本  y_i 代表数据值
    return X_b_i.T.dot(X_b_i.dot(theta) - y_i) * 2.

In [7]:
def sgd(X_b, y, initial_theta, n_iters):
    a = 5
    b = 50
    def learning_rate(iter):
        return a/(b + iter)
    # 现在是随机选一个点 可能这2个点隔的很近 所以epsilon 已经没有意义了
    theta = initial_theta
    for cur_iter in range(n_iters):
        rand_i = np.random.randint(len(X_b))
        gradient = dJ_sgd(theta, X_b[rand_i], y[rand_i])
        theta = theta - learning_rate(cur_iter) * gradient
    return theta
    

In [8]:
%%time
X_b = np.hstack((np.ones((len(X), 1)), X))
initial_theta = np.zeros(X_b.shape[1])
# 这里只是举例取1/3的数据
theta = sgd(X_b, y, initial_theta, n_iters=len(X_b)//3)

Wall time: 250 ms


In [9]:
theta

array([2.98087918, 3.97911084])

# 使用我们自己的sgd

In [10]:
from playML.LinearRegression import LinearRegression

In [11]:
lin_reg = LinearRegression()

In [12]:
lin_reg.fit_sgd(X, y, n_iters=3)

In [13]:
lin_reg.coef_

array([3.99499373])

In [14]:
lin_reg.interception_

2.9961559846119195

# 使用真实数据

In [15]:
from sklearn import datasets
boston = datasets.load_boston()

In [16]:
X = boston.data
y = boston.target
X = X[y<50]
y = y[y<50]

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [19]:
# normalization
from sklearn.preprocessing import StandardScaler

In [20]:
standardScaler = StandardScaler()

In [21]:
standardScaler.fit(X_train)
X_train_std = standardScaler.transform(X_train)

In [22]:
X_test_std = standardScaler.transform(X_test)

In [23]:
lin_reg = LinearRegression()
%time lin_reg.fit_sgd(X_train_std, y_train, n_iters=2)
print(lin_reg.score(X_test_std, y_test))

Wall time: 4 ms
0.41567098216317067


In [24]:
lin_reg = LinearRegression()
%time lin_reg.fit_sgd(X_train_std, y_train, n_iters=20)
print(lin_reg.score(X_test_std, y_test))

Wall time: 32 ms
0.8011412628426213


In [25]:
lin_reg = LinearRegression()
%time lin_reg.fit_sgd(X_train_std, y_train, n_iters=100)
print(lin_reg.score(X_test_std, y_test))

Wall time: 157 ms
0.7953879813032926


# sklearn sgd

In [26]:
from sklearn.linear_model import SGDRegressor

In [27]:
sgd_reg = SGDRegressor(max_iter = 100)
%time sgd_reg.fit(X_train_std, y_train)
sgd_reg.score(X_test_std, y_test)

Wall time: 4 ms


0.8001807065216349

### 可以看出sklearn中的sgd 非常的块
## 关于梯度的调试
#### 通过求临近点的斜率来近似查看梯度是否正确
![e1](img/e1.png)
![e2](img/e2.png)

In [28]:
import numpy as np
import matplotlib.pyplot as plt

In [29]:
#np.random.seed = 666
# 1000 * 10 的矩阵
X = np.random.random(size=(1000,10))

In [30]:
# 真正的theta  10个特征 加上截距
true_theta = np.arange(1, 12, dtype=float)

In [31]:
X_b = np.hstack([np.ones((len(X), 1)), X])
y = X_b.dot(true_theta) 
#+ np.random.normal(size=1000)

In [32]:
X.shape

(1000, 10)

In [33]:
true_theta

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [34]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(X_b)
    except:
        return float("inf")

In [35]:
def dJ_math(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) *2. /len(y)

In [36]:
def dJ_debug(theta, X_b, y, epsilon = 0.01):
    res = np.empty(len(theta))
    for i in range(len(theta)):
        theta_1 = theta.copy()
        theta_2 = theta.copy()
        theta_1[i] += epsilon
        theta_2[i] -= epsilon
        res[i] = (J(theta_1, X_b, y) - J(theta_2, X_b,y))/(2*epsilon)
    return res

In [37]:
def gradient_descent(dJ, X_b, y, initial_theta, eta, n_iters=1e4, epsilon=1e-2):
    theta = initial_theta
    cur_iter = 0
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        
        cur_iter +=1
    return theta

In [38]:
X_b = np.hstack((np.ones((len(X), 1)), X))
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
%time theta = gradient_descent(dJ_debug, X_b, y, initial_theta, eta)

Wall time: 112 ms


In [39]:
theta

array([8.2551353 , 3.15797698, 3.60345071, 3.79212073, 4.61225969,
       4.83885579, 5.1554052 , 5.96976643, 6.10127475, 6.60692638,
       7.04504714])

In [40]:
X_b = np.hstack((np.ones((len(X), 1)), X))
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
%time theta = gradient_descent(dJ_math, X_b, y, initial_theta, eta)
print(theta)

Wall time: 29 ms
[8.2551353  3.15797698 3.60345071 3.79212073 4.61225969 4.83885579
 5.1554052  5.96976643 6.10127475 6.60692638 7.04504714]


In [41]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(max_iter=5)

In [42]:
reg.fit(X, y)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=5, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [43]:
reg.coef_

array([2.62554924, 3.4433951 , 3.83991962, 4.54775718, 4.74824703,
       5.34638131, 6.21657188, 6.08042501, 7.02927681, 7.47806428])

In [44]:
reg.intercept_

array([7.9308028])

In [45]:
np.random.normal(size=10)

array([-0.4728813 ,  2.19034926, -0.50771146,  0.17360658,  1.5301316 ,
       -0.70826627,  1.18107397, -0.49045393, -0.32292588,  1.84615109])

## 总结

![sum1](img/sum1.png)